In [19]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob

In [20]:
IMG_SIZE = [224, 224]

In [21]:
train_directory="../input/100-bird-species/train"
test_directory="../input/100-bird-species/test"
val_directory="../input/100-bird-species/valid"

In [22]:
vgg = VGG16(input_shape=IMG_SIZE + [3], weights="imagenet", include_top=False)

for layer in vgg.layers:
  layer.trainable = False

In [23]:
folders = glob("../input/100-bird-species/train/*")
len(folders)

In [24]:
x = Flatten()(vgg.output)

prediction = Dense(len(folders), activation="softmax")(x)

In [25]:
model = Model(inputs=vgg.input, outputs=prediction)

model.summary()

In [26]:
model.compile(
loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [27]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_directory,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = "categorical")

test_set = test_datagen.flow_from_directory(test_directory,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = "categorical")


In [28]:
print(len(training_set))
print(len(test_set))

In [29]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks = [earlystopping]
)

In [30]:
plt.plot(r.history["loss"], label="train loss")
plt.plot(r.history["val_loss"], label="val loss")
plt.legend()
plt.show()

In [31]:
plt.plot(r.history["accuracy"], label="train acc")
plt.plot(r.history["val_accuracy"], label="val acc")
plt.legend()
plt.show()

In [32]:
model.save("model")